# 🚀 Emotion Classification with BERT-based Models

This notebook reproduces the results for **Project 2: Emotion Classification**.
It covers setup, downloading pre-trained weights, training, evaluation, and compression benchmarks.

### ⚠️ Hardware Requirement: GPU Required
**Please ensure you are connected to a GPU Runtime.**
- Go to `Runtime` > `Change runtime type` > `Hardware accelerator` > Select **T4 GPU** (or A100/V100).
- **Do NOT use TPU**: The compression library `bitsandbytes` (used for 4-bit quantization) currently relies on CUDA kernels and is not compatible with Colab TPUs.

## 1. Setup & Installation

In [ ]:
# 1. Clone the repository
!git clone https://github.com/Roeyx/NLP.git

# 2. Enter the project directory
%cd NLP/Project2

# 3. Install dependencies
!pip install -r requirements.txt

# 4. Create outputs directory
!mkdir -p outputs

## 2. Download Pre-trained Weights
We download the trained checkpoints to skip the training phase.

In [ ]:
# Download Best Original Model (CardiffRoBERTa - FP32)
!gdown 1hU4AA1mIE6fJHYEYUB0lgwBnyiESjoFK -O outputs/best_CardiffRoBERTa.pt

# Download Best Compressed Model (Quantized NF4)
!gdown 1bYhpRRKBpJu3RNcOG753ss4SMXMeqwPA -O outputs/best_CardiffRoBERTa_quantized_nf4.pt

## 3. Training Demo
Train a fresh model from scratch (using ModernBERT for efficiency).

In [ ]:
!python emotion_classifier.py --mode train --model ModernBERT

## 4. Evaluation (Reproduce Results)
Evaluate the downloaded models on the validation set to verify Accuracy and F1 scores.

In [ ]:
print("=== Evaluating Original Model (FP32) ===")
!python emotion_classifier.py --mode evaluate --weights outputs/best_CardiffRoBERTa.pt --val validation.csv

In [ ]:
print("=== Evaluating Compressed Model (Quantized NF4) ===")
!python emotion_classifier.py --mode evaluate --weights outputs/best_CardiffRoBERTa_quantized_nf4.pt --val validation.csv

## 5. Inference on Custom Test Data
Upload your own `test.csv` file (must contain a 'text' column) to generate predictions.

**Note:**
- Predictions will be saved to `<filename>_predictions.csv`.
- The file contains the original text plus `predicted_label` (0-5) and `predicted_emotion`.

In [ ]:
from google.colab import files
import os

print("Please upload your test.csv file:")
uploaded = files.upload()

filename = list(uploaded.keys())[0]
print(f"Uploaded: {filename}")

In [ ]:
# Run inference using the Best Original Model
!python emotion_classifier.py --mode inference --weights outputs/best_CardiffRoBERTa.pt --test_csv "{filename}"

output_file = f"{os.path.splitext(filename)[0]}_predictions.csv"
print(f"\n✅ Predictions saved to: {output_file}")

# Display first few predictions
!head -n 5 "{output_file}"

# Download the results
files.download(output_file)

## 6. Compression Analysis
Run the full compression benchmark suite (Pruning, INT8, NF4, Combined) to generate the comparison report.

**Note:** This step requires the GPU runtime for `bitsandbytes` quantization.

In [ ]:
!python emotion_classifier.py --mode compress --weights outputs/best_CardiffRoBERTa.pt --val validation.csv